In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import math
import numpy as np
import pandas as pd
import random
import string
import re
import os
from collections import Counter,OrderedDict
from tqdm.auto import tqdm

import nltk
from nltk.tokenize import word_tokenize
# from nltk.lm import Vocabulary
from sklearn.model_selection import train_test_split

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
class SeqDataset(Dataset):
    def __init__(self,en_seqs,fr_seqs):
        super().__init__()
        self.en_seqs = en_seqs
        self.fr_seqs = fr_seqs

    def __len__(self):
        return len(self.en_seqs)

    def __getitem__(self, index):
        encoder_input = torch.tensor(self.en_seqs[index]).long()
        decoder_input = torch.tensor(self.fr_seqs[index][:-1]).long()
        answer = torch.tensor(self.fr_seqs[index][1:]).long()     #right shift
        return encoder_input, decoder_input, answer

def collate_fn(batch):
    encoder_ip = []
    decoder_ip = []
    answer = []
    for e, d, a in batch:
        encoder_ip.append(e)
        decoder_ip.append(d)
        answer.append(a)

    encoder_ip = nn.utils.rnn.pad_sequence(encoder_ip, batch_first= True, padding_value= 16222)
    decoder_ip = nn.utils.rnn.pad_sequence(decoder_ip, batch_first= True, padding_value= 31540)
    answer = nn.utils.rnn.pad_sequence(answer, batch_first= True, padding_value= 31540)
    return encoder_ip, decoder_ip, answer


In [ ]:
import torch

# Load training dataset with weights_only=True
train_data = torch.load('/kaggle/input/en-fr-translation/train_dataset.pt', weights_only=True)
loaded_trainset = SeqDataset(train_data['en_seqs'], train_data['fr_seqs'])

# Load testing dataset with weights_only=True
test_data = torch.load('/kaggle/input/en-fr-translation/test_dataset.pt', weights_only=True)
loaded_testset = SeqDataset(test_data['en_seqs'], test_data['fr_seqs'])


In [ ]:
batch_size=16

In [ ]:
trainloader = DataLoader(loaded_trainset, batch_size= batch_size, shuffle= True, collate_fn= collate_fn)
testLoader = DataLoader(loaded_testset, batch_size= batch_size, shuffle= False, collate_fn= collate_fn)

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self, dim_model, dropout_p, max_len):
    super().__init__()
    self.dropout = nn.Dropout(dropout_p)

    pe = torch.zeros(max_len, dim_model)
    ps = torch.arange(0, max_len, dtype=torch.float).view(-1, 1) # 0, 1, 2, 3, 4, 5
    div_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) # 1000^(2i/dim_model)

    # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
    pe[:, 0::2] = torch.sin(ps * div_term)

    # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
    pe[:, 1::2] = torch.cos(ps * div_term)

    pe = pe.unsqueeze(0).transpose(0, 1)
    self.register_buffer("pe", pe)

  def forward(self, token_embedding: torch.tensor) -> torch.tensor:
    return self.dropout(token_embedding + self.pe[:token_embedding.size(0), :])

In [ ]:
class Transformer(nn.Module):
  def __init__(self, ip_dim, out_dim, d_model, num_heads, num_encoder_layers, num_decoder_layers, dropout_p, src_pad_idx=None, tgt_pad_idx=None):
    super().__init__()

    self.model_type = "Transformer"
    self.d_model = d_model
    self.src_pad_idx = src_pad_idx
    self.tgt_pad_idx = tgt_pad_idx

    self.positional_en = PositionalEncoding(dim_model=d_model, dropout_p=dropout_p, max_len=5000)
    self.embedding_ip = nn.Embedding(ip_dim, d_model)
    self.embedding_out = nn.Embedding(out_dim, d_model)
    self.transformer = nn.Transformer(
      d_model=d_model,
      nhead=num_heads,
      num_encoder_layers=num_encoder_layers,
      num_decoder_layers=num_decoder_layers,
      dropout=dropout_p,
      batch_first=True
    )
    self.out = nn.Linear(d_model, out_dim)

  def forward(self, src, tgt):
    src_pad_mask = self.create_pad_mask(src, pad_token=self.src_pad_idx)
    tgt_pad_mask = self.create_pad_mask(tgt, pad_token=self.tgt_pad_idx)

    # (batch_size, sequence length, d_model)
    src = self.embedding_ip(src) * math.sqrt(self.d_model)
    tgt = self.embedding_out(tgt) * math.sqrt(self.d_model)
    src = self.positional_en(src)
    tgt = self.positional_en(tgt)


    tgt_mask = self.get_tgt_mask(tgt.size(1)).to(src.device)

    # (sequence length, batch_size, num_tokens)
    transformer_out = self.transformer(src, tgt, tgt_mask=tgt_mask, src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask)
    out = self.out(transformer_out)

    return out

  def get_tgt_mask(self, size) -> torch.tensor:
    mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
    mask = mask.float()
    mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
    mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0
    return mask

  def create_pad_mask(self, matrix: torch.tensor, pad_token: int) -> torch.tensor:
    return (matrix == pad_token)

In [ ]:
en_vocab = 16224
fr_vocab = 31542

In [ ]:
# ip_dim = len(en_vocab)
# out_dim = len(fr_vocab)
ip_dim = en_vocab
out_dim = fr_vocab
d_model = 512
num_enc_layers = 2
num_dec_layers = 2
n_heads= 8
dropout_p = 0.2
src_pad_idx= 16222
tgt_pad_idx= 31540

In [ ]:
model = Transformer(ip_dim=ip_dim, out_dim=out_dim, d_model=d_model, num_heads=n_heads, num_encoder_layers=num_enc_layers,
                    num_decoder_layers=num_dec_layers, dropout_p=dropout_p, src_pad_idx=src_pad_idx, tgt_pad_idx=tgt_pad_idx).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index= tgt_pad_idx)
optimizer = optim.AdamW(model.parameters(), lr= 3e-4)
model

In [ ]:
!pip install torchinfo

In [ ]:
import torchinfo
torchinfo.summary(model)

In [ ]:
def train_epoch(model, dataloader, loss_fn, optimizer, device=device):
  epoch_loss = 0
  correct_predictions = 0
  total_predictions = 0

  model.train()
  for batch in tqdm(dataloader, desc="Training", leave=False):
    src, tgt, tgt_y = batch
    src, tgt, tgt_y = src.to(device), tgt.to(device), tgt_y.to(device)

    optimizer.zero_grad()

    output = model(src, tgt)

    loss = loss_fn(output.view(-1, output.size(-1)), tgt_y.contiguous().view(-1))
    loss = loss.to(device)

    loss.backward()

    optimizer.step()

    epoch_loss += loss.item()

    preds = output.argmax(dim=-1)
    correct_predictions += (preds == tgt_y).sum().item()
    total_predictions += tgt_y.numel()

  avg_epoch_loss = epoch_loss / len(dataloader)
  accuracy = correct_predictions / total_predictions
  return avg_epoch_loss, accuracy

In [ ]:
def test_epoch(model, dataloader, loss_fn, device = device):
  epoch_loss = 0
  correct_predictions = 0
  total_predictions = 0

  model.eval()
  with torch.no_grad():
    for batch in tqdm(dataloader, desc="Testing", leave=False):
      src, tgt, tgt_y = batch
      src, tgt, tgt_y = src.to(device), tgt.to(device), tgt_y.to(device)

      output = model(src, tgt)

      loss = loss_fn(output.view(-1, output.size(-1)), tgt_y.contiguous().view(-1))
      loss = loss.to(device)

      epoch_loss += loss.item()

      preds = output.argmax(dim=-1)
      correct_predictions += (preds == tgt_y).sum().item()
      total_predictions += tgt_y.numel()

  avg_epoch_loss = epoch_loss / len(dataloader)
  accuracy = correct_predictions / total_predictions
  return avg_epoch_loss, accuracy

In [ ]:
def save_model(model, epoch, train_acc, train_loss, val_acc, val_loss, directory='/kaggle/working'):
  filename = (f"ep-{epoch+1:03d}-train_acc-{train_acc:.4f}-train_loss-{train_loss:.4f}-"
              f"val_acc-{val_acc:.4f}-val_loss-{val_loss:.4f}.pth")
  filepath = os.path.join(directory, filename)

  torch.save(model.state_dict(), filepath)
  print(f"Model saved to {filepath}")

In [ ]:
# model.load_state_dict(torch.load('/kaggle/input/state-dict/ep-050-train_acc-0.4313-train_loss-0.5132-val_acc-0.3897-val_loss-1.4604.pth'))

In [ ]:
epochs = 100
for epoch in range(epochs):
  train_loss, train_accuracy = train_epoch(model, trainloader, loss_fn, optimizer, device)
  test_loss, test_accuracy = test_epoch(model, testLoader, loss_fn, device)

  print(f'Epoch {epoch+1}/{epochs}')
  print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
  print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')
  if(epoch+1) % 5 == 0:
    save_model(model, epoch, train_accuracy, train_loss, test_accuracy, test_loss)
    print(f'Model saved at {epoch+1}')